In [ ]:
pip install fastai -U

In [2]:
pip install sentencepiece!=0.1.90,!=0.1.91

     |████████████████████████████████| 1.0MB 6.6MB/s 


In [3]:
from IPython.display import display, HTML

In [4]:
import fastai
fastai.__version__
from fastai.text.all import *

In [5]:
### Language Model
## Getting the data 
path = untar_data(URLs.IMDB)

In [6]:
# Looking inside the directories
path.ls()

(#7) [Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/test')]

In [7]:
# We only need train, test and unsup sub-folder
files = get_text_files(path, folders=['train', 'test', 'unsup'])

In [8]:
## Word Tokenization
txt = files[0].open().read(); txt[:75]

'WWE Armageddon, December 17, 2006 -- Live from Richmond Coliseum, Richmond,'

In [9]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#1106) ['WWE','Armageddon',',','December','17',',','2006','--','Live','from','Richmond','Coliseum',',','Richmond',',','VA','<','br','/><br','/>Kane','vs.','MVP','in','an','Inferno','match',':','So','this','is'...]


In [10]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [11]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#1278) ['xxbos','xxup','wwe','xxmaj','armageddon',',','xxmaj','december','17',',','2006','--','xxmaj','live','from','xxmaj','richmond','xxmaj','coliseum',',','xxmaj','richmond',',','xxup','va','\n\n','xxmaj','kane','vs.','xxup','mvp'...]


In [12]:
coll_repr(tkn("&copy;   Fast.ai WWW.fast.ai/INDEX   "), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index'...]"

In [13]:
## Sub-word tokenization
txts = L(o.open().read() for o in files[:2000])

In [14]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return " ".join(first(sp([txt]))[:40])

In [15]:
subword(1000)

'▁W W E ▁A r m age d d on , ▁De ce m ber ▁1 7 , ▁2 0 0 6 ▁ -- ▁Li ve ▁from ▁R i ch mon d ▁C ol is e um , ▁R i'

In [16]:
subword(200)

'▁ W W E ▁A r m a g ed d on , ▁ D e ce m b er ▁ 1 7 , ▁ 2 0 0 6 ▁ - - ▁ L i ve ▁f ro m ▁'

In [17]:
subword(10000)

'▁WW E ▁Ar m aged don , ▁ December ▁17 , ▁2006 ▁-- ▁Live ▁from ▁Rich mond ▁Col is e um , ▁Rich mond , ▁V A ▁< br ▁/> < br ▁/> K ane ▁vs . ▁M V P'

In [18]:
## Numericalization

toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#1278) ['xxbos','xxup','wwe','xxmaj','armageddon',',','xxmaj','december','17',',','2006','--','xxmaj','live','from','xxmaj','richmond','xxmaj','coliseum',',','xxmaj','richmond',',','xxup','va','\n\n','xxmaj','kane','vs.','xxup','mvp'...]


In [19]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#1278) ['xxbos','xxup','wwe','xxmaj','armageddon',',','xxmaj','december','17',','...]

In [20]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#2064) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','i','it'...]"

In [21]:
nums = num(toks)[:20]; nums

tensor([   2,    7,  972,    8,  973,   10,    8,    0,    0,   10, 1188,  297,
           8,  424,   48,    8,    0,    8,    0,   10])

In [22]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxup wwe xxmaj armageddon , xxmaj xxunk xxunk , 2006 -- xxmaj live from xxmaj xxunk xxmaj xxunk ,'

In [23]:
## Batching the text data
## Example that is not feasible

stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs, seq_len = 6, 15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [24]:
# Data is randomized and a continuous stream is produced.
# This gets fed to the NN in batches. Model has a mechanism 
# that keeps track of the order. All this is done in the background.

nums200 = toks200.map(num)

In [25]:
dl = LMDataLoader(nums200)

In [26]:
# Confirming that batch has what we anticipate

x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [27]:
## Looking at the first row of the independent variable

" ".join(num.vocab[o] for o in x[0][:20])

'xxbos xxup wwe xxmaj armageddon , xxmaj xxunk xxunk , 2006 -- xxmaj live from xxmaj xxunk xxmaj xxunk ,'

In [28]:
## Looking at the first row of the dependent variable which should be the same thing
# ofset by one. 

" ".join(num.vocab[o] for o in y[0][:20])

'xxup wwe xxmaj armageddon , xxmaj xxunk xxunk , 2006 -- xxmaj live from xxmaj xxunk xxmaj xxunk , xxmaj'

In [41]:
## Done with processing.
## Training a text classifier
## Using TextBlock to create a language model

get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [36]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos i really thought that this movie was superb . xxmaj not only is the history correct , but the style is sumptuous and yet intimate . i was a fan of xxmaj emily xxmaj blunt 's portrayal of xxmaj victoria and how she kept her spirit even though she was forced into a virtual exile while in her youth . xxmaj blunt depicts the charismatic and sometimes dogmatic manner that xxmaj xxunk became famous ( perhaps infamous ) for","i really thought that this movie was superb . xxmaj not only is the history correct , but the style is sumptuous and yet intimate . i was a fan of xxmaj emily xxmaj blunt 's portrayal of xxmaj victoria and how she kept her spirit even though she was forced into a virtual exile while in her youth . xxmaj blunt depicts the charismatic and sometimes dogmatic manner that xxmaj xxunk became famous ( perhaps infamous ) for ."
1,"- apocalypse ' story where mankind is reduced to living an almost stone - age existence by a global disaster , but this time the apocalypse is brought about not by nuclear weapons but a race of dragons unleashed from the depths of the earth by over - ambitious mining ! xxup ok , it 's a pretty silly concept , but the whole thing is handled with panache . xxmaj director xxmaj rob ( x - files ) xxmaj","apocalypse ' story where mankind is reduced to living an almost stone - age existence by a global disaster , but this time the apocalypse is brought about not by nuclear weapons but a race of dragons unleashed from the depths of the earth by over - ambitious mining ! xxup ok , it 's a pretty silly concept , but the whole thing is handled with panache . xxmaj director xxmaj rob ( x - files ) xxmaj bowman"


In [37]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_multi=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
# learn.fit_one_cycle(1, 2e-2)

In [ ]:
# learn.save('/content/epoch1')

Path('/content/epoch1.pth')

In [38]:
# Loading the saved model
learn = learn.load("/content/drive/My Drive/Models/1epoch")

In [ ]:
# Unfreeze the pretrained layers and train for a few epochs
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
# dls_clas = DataBlock(
#     blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
#     get_y = parent_label, get_items=partial(get_text_files, folders=['train', 'text']),
#     splitter= GrandparentSplitter(valid_name='test')
# ).dataloaders(path, path=path, bs=128, seq_len=72)